In [1]:
# adapted from Julia Lane course and https://stackabuse.com/python-for-nlp-topic-modeling/

import pandas as pd
import numpy as np
import re
import nltk
import time
from datetime import datetime

In [2]:
# cd to the directory with project data
# %cd '/path/to/your/data'

In [3]:
start_time = time.time()
# 10 years of project data
fiscal_years = ['2010','2011','2012','2013','2014','2015','2016','2017','2018']
prefix = 'FedRePORTER_PRJ_C_FY'
suffix = '.csv'

# initialize dataframe with fy09 data
file = 'FedRePORTER_PRJ_C_FY2009.csv'
print('Reading in ' + file)
projects_df = (pd.read_csv(file,skipinitialspace=True,encoding='utf-8'))

# concatenate 10 years of data
for year in fiscal_years:
    file = prefix + year + suffix
    print('Reading in ' + file)
    projects_df = projects_df.append(pd.read_csv(file, skipinitialspace=True, encoding='utf-8'), ignore_index=True)

# new variable is 1 for rows with opioid in project term column
projects_df['opioid'] = np.where(
    projects_df['PROJECT_TERMS'].str.contains("opioid",case=False, na=False), 1, '')

# create a numeric version of our flag
projects_df['opioid_num'] = pd.to_numeric(projects_df['opioid'])

elapsed_time = time.time() - start_time
time.strftime("%H:%M:%S", time.gmtime(elapsed_time))


Reading in FedRePORTER_PRJ_C_FY2009.csv


/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/Wagner/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (12,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Reading in FedRePORTER_PRJ_C_FY2010.csv
Reading in FedRePORTER_PRJ_C_FY2011.csv
Reading in FedRePORTER_PRJ_C_FY2012.csv
Reading in FedRePORTER_PRJ_C_FY2013.csv
Reading in FedRePORTER_PRJ_C_FY2014.csv
Reading in FedRePORTER_PRJ_C_FY2015.csv
Reading in FedRePORTER_PRJ_C_FY2016.csv


/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/Wagner/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Reading in FedRePORTER_PRJ_C_FY2017.csv
Reading in FedRePORTER_PRJ_C_FY2018.csv


'00:01:21'

In [3]:
# cd to the directory with abstracts data
# %cd '/path/to/your/data'

In [5]:
start_time = time.time()

# 10 years of abstracts data
fiscal_years = ['2010','2011','2012','2013','2014','2015','2016','2017','2018']
prefix = 'FedRePORTER_PRJABS_C_FY'
suffix = '.csv'

# initialize dataframe with fy09 data
file = 'FedRePORTER_PRJABS_C_FY2009.csv'
print('Reading in ' + file)
abstracts_df = (pd.read_csv(file,skipinitialspace=True,encoding='utf-8'))

for year in fiscal_years:
    file = prefix + year + suffix
    print('Reading in ' + file)
    abstracts_df = abstracts_df.append(pd.read_csv(file, skipinitialspace=True, encoding='utf-8'), ignore_index=True)

elapsed_time = time.time() - start_time
time.strftime("%H:%M:%S", time.gmtime(elapsed_time))

Reading in FedRePORTER_PRJABS_C_FY2009.csv
Reading in FedRePORTER_PRJABS_C_FY2010.csv
Reading in FedRePORTER_PRJABS_C_FY2011.csv
Reading in FedRePORTER_PRJABS_C_FY2012.csv
Reading in FedRePORTER_PRJABS_C_FY2013.csv
Reading in FedRePORTER_PRJABS_C_FY2014.csv
Reading in FedRePORTER_PRJABS_C_FY2015.csv
Reading in FedRePORTER_PRJABS_C_FY2016.csv
Reading in FedRePORTER_PRJABS_C_FY2017.csv
Reading in FedRePORTER_PRJABS_C_FY2018.csv


'00:01:07'

In [6]:
# Affilated terms from https://en.wikipedia.org/wiki/Opioid
opioid_terms = ['opioid','opiate','morphine','heroin','percocet',
                'vicoprofen','dextromethorphan','loperamide',
                'naloxegol',
                'hydrocodone','oxycodone','fentanyl','naloxone',
                'analgesics', 'carfentanil','benzodiazepines',
                'narcotic','opium','cocaine','codeine',
                'pain relief','cancer pain','anesthesia','chronic pain',
                'nerve pain','fibromyalgia',
                'overdose','addiction','withdrawal','dependence',
                'recreational use','euphoria','tolerance',
                'controlled substance','over-prescription',
                'peripheral nervous system','psychoactive','agonist',
                'antagonist','blood-brain']

In [7]:
start_time = time.time()

# keep only variables needed and remove missing values
abstracts_10yrs = abstracts_df.dropna()

# cut the end coding that interferes with cleaning script
abstracts_10yrs = abstracts_10yrs[0:-1]

#get rid of the punctuations and set all characters to lowercase
nonchars = re.compile( r'\W+|\d+' )

def clean(text):
    return re.sub(nonchars, " ", text).lower()

abstracts_10yrs['cleanText'] = abstracts_10yrs['ABSTRACT'].apply(clean)

elapsed_time = time.time() - start_time
time.strftime("%H:%M:%S", time.gmtime(elapsed_time))

'00:09:24'

In [8]:
start_time = time.time()

# Count the appearances of our defined terms in each abstract
def countTerm(text):
    return len(re.findall(term,text))

for term in opioid_terms:
    abstracts_10yrs[term] = abstracts_10yrs['cleanText'].apply(countTerm)
    
# sum of all term frequencies by abstract
abstracts_10yrs['sumTermCounts'] = abstracts_10yrs[opioid_terms].sum(axis=1)

print('The number of abstracts with three or more opioid terms is: ' +
     str(abstracts_10yrs[abstracts_10yrs['sumTermCounts']>2].shape[0]))

elapsed_time = time.time() - start_time
time.strftime("%H:%M:%S", time.gmtime(elapsed_time))

The number of abstracts with three or more opioid terms is: 32647


'02:35:45'

In [9]:
start_time = time.time()

# merge projects and abstracts by PROJECT_ID
merged_df_10yrs = pd.merge(projects_df, abstracts_10yrs, on='PROJECT_ID')

elapsed_time = time.time() - start_time
time.strftime("%H:%M:%S", time.gmtime(elapsed_time))

'00:00:23'

In [10]:
# define compareTags function

wikiThreshold = 2;

def compareTags(row):
    if (row['opioid_num'] == 1) & (row['sumTermCounts'] > wikiThreshold):
        return 'both'
    if row['opioid_num'] == 1:
        return 'explicitOnly'
    if row['sumTermCounts'] > wikiThreshold:
        return 'wikiOnly'
    return 'neither'

In [11]:
start_time = time.time()

# compare 10yr data tags
merged_df_10yrs['tagCompare'] = merged_df_10yrs.apply(compareTags, axis=1)

print('Number of projects')
merged_df_10yrs['tagCompare'].value_counts()

elapsed_time = time.time() - start_time
time.strftime("%H:%M:%S", time.gmtime(elapsed_time))

Number of projects


'00:01:44'

In [12]:
print('Cost sums')
print('Neither:       ' + str(merged_df_10yrs.FY_TOTAL_COST[merged_df_10yrs.tagCompare == 'neither'].sum()))
print('Both:          ' + str(merged_df_10yrs.FY_TOTAL_COST[merged_df_10yrs.tagCompare == 'both'].sum()))
print('Explicit only: ' + str(merged_df_10yrs.FY_TOTAL_COST[merged_df_10yrs.tagCompare == 'explicitOnly'].sum()))
print('Wiki only:     ' + str(merged_df_10yrs.FY_TOTAL_COST[merged_df_10yrs.tagCompare == 'wikiOnly'].sum()))

Cost sums
Neither:       339529264262.0
Both:          2302111743.0
Explicit only: 954076329.0
Wiki only:     9840458684.0


In [4]:
# cd to the directory where you want to save your data
# %cd '/path/to/your/data'

In [15]:
# export csv with progress so far
start_time = time.time()

merged_df_10yrs.to_csv('opioidAnalyticalData.csv',encoding='utf-8-sig')

elapsed_time = time.time() - start_time
print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))

00:17:31
